# OR hw2

In [1]:
import pandas as pd
import openpyxl
from gurobipy import *

## Gurobi

### Constant Data – part (b) instance

In [2]:
# 有任何問題歡迎寄信詢問助教 b07705036@ntu.edu.tw
I = 3 # machine num
J = 10 # job num
machines = range(0, I)
jobs = range(0, J)

In [3]:
# process time for job j
p = [3, 6, 5, 4, 1, 8, 5, 12, 7, 6]

# benefit for job j
b = [5, 8, 4, 6, 3, 7, 6, 9, 5, 8]

# machine capacity
K = 15

### Constant Data – part (c) instance

In [4]:
I = 4 # machine num
J = 25 # job num
machines = range(0, I)
jobs = range(0, J)

In [5]:
# process time for job j
p = [3, 6, 7, 5, 4, 2, 6, 3, 5, 8, 10, 2, 4, 7, 1, 5, 8, 3, 6, 4, 12, 4, 8, 4, 7]

# benefit for job j
b = [5, 8, 12, 4, 6, 6, 3, 7, 6, 15, 9, 5, 8, 10, 1, 5, 3, 7, 12, 14, 5, 8, 9, 8, 10]

# machine capacity
K = 15

### Decision variable

In [6]:
Q3 = Model('Q3')

# x_ij = 0 if job j is assigned to machine i
x = {}
for i in machines:
    x[i] = {}
    for j in jobs:
        x[i][j] = Q3.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(j))

w = Q3.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")

Q3.update()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-05-07
Using license file /Users/linyuanting/gurobi.lic


### Setting the objective function

In [7]:
Q3.setObjective(w, GRB.MAXIMIZE)

### Add constraints

In [8]:
Q3.addConstrs((w <= quicksum(b[j] * x[i][j] for j in jobs) for i in machines), 'least benefit machine')
Q3.addConstrs((quicksum(x[i][j] for i in machines) <= 1 for j in jobs), 'each job assigned to exactly one machine')
Q3.addConstrs((quicksum(p[j] * x[i][j] for j in jobs) <= K for i in machines), 'machine capacity')

Q3.update()

In [9]:
Q3.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 33 rows, 101 columns and 304 nonzeros
Model fingerprint: 0xa40dd69d
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve time: 0.01s
Presolved: 33 rows, 101 columns, 304 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   4.000000e+30   8.000000e+00      0s
      40    3.0392857e+01   0.000000e+00   0.000000e+00      0s

Solved in 40 iterations and 0.01 seconds
Optimal objective  3.039285714e+01


In [10]:
upper_bound = Q3.objVal
print("objective value =", Q3.objVal)

objective value = 30.39285714285714


## Results

In [11]:
x_opt = {}

for i in machines:
    x_opt[i] = {}
    for j in jobs:
        x_opt[i][j] = {}

for var in Q3.getVars():
    index = [i for i in var.varName[2:].split(',')]
    if var.varName[0] == 'x':
        x_opt[int(index[0])][int(index[1])] = var.x

In [12]:
pd.DataFrame(x_opt)

,0,1,2,3
0,0.000000,0.000000,1.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000
2,0.000000,1.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,1.000000,0.000000,0.000000,0.000000
5,1.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,1.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,1.000000,0.000000


## Heuristic – HBF (highest benefit first)

### Constant Data – part (b) instance

In [13]:
I = 3 # machine num
J = 10 # job num
machines = range(0, I)
jobs = range(0, J)

In [14]:
# process time for job j
p = [3, 6, 5, 4, 1, 8, 5, 12, 7, 6]

# benefit for job j
b = [5, 8, 4, 6, 3, 7, 6, 9, 5, 8]

# machine capacity
K = 15

### Constant Data – part (c) instance

In [15]:
I = 4 # machine num
J = 25 # job num
machines = range(0, I)
jobs = range(0, J)

In [16]:
# process time for job j
p = [3, 6, 7, 5, 4, 2, 6, 3, 5, 8, 10, 2, 4, 7, 1, 5, 8, 3, 6, 4, 12, 4, 8, 4, 7]

# benefit for job j
b = [5, 8, 12, 4, 6, 6, 3, 7, 6, 15, 9, 5, 8, 10, 1, 5, 3, 7, 12, 14, 5, 8, 9, 8, 10]

# machine capacity
K = 15

In [17]:
# [b, p, index] * job num
job_list = list(zip(b, p, range(1, J+1)))
job_list.sort(key=lambda x: (-x[0], x[2]))
print(job_list)

[(15, 8, 10), (14, 4, 20), (12, 7, 3), (12, 6, 19), (10, 7, 14), (10, 7, 25), (9, 10, 11), (9, 8, 23), (8, 6, 2), (8, 4, 13), (8, 4, 22), (8, 4, 24), (7, 3, 8), (7, 3, 18), (6, 4, 5), (6, 2, 6), (6, 5, 9), (5, 3, 1), (5, 2, 12), (5, 5, 16), (5, 12, 21), (4, 5, 4), (3, 6, 7), (3, 8, 17), (1, 1, 15)]


In [18]:
# [current benefit, current process time, assigned job list, index] * machine num
machine_list = [[0, 0, [], i+1] for i in machines]
for j in jobs:
    for i in machines:
        if machine_list[i][1] + job_list[j][1] <= K:
            machine_list[i][0] += job_list[j][0]
            machine_list[i][1] += job_list[j][1]
            machine_list[i][2].append(job_list[j][2])
            machine_list.sort(key=lambda x: (x[0], x[3]))
            print('b =', [machine_list[k][0] for k in machines], 'p =', [machine_list[k][1] for k in machines])
            break

b = [0, 0, 0, 15] p = [0, 0, 0, 8]
b = [0, 0, 14, 15] p = [0, 0, 4, 8]
b = [0, 12, 14, 15] p = [0, 7, 4, 8]
b = [12, 12, 14, 15] p = [7, 6, 4, 8]
b = [12, 14, 15, 22] p = [6, 4, 8, 14]
b = [14, 15, 22, 22] p = [4, 8, 14, 13]
b = [15, 22, 22, 23] p = [8, 14, 13, 14]
b = [22, 22, 23, 23] p = [14, 13, 14, 14]
b = [22, 23, 23, 28] p = [14, 14, 14, 15]
b = [23, 23, 23, 28] p = [14, 14, 15, 15]


In [19]:
machine_list

[[23, 14, [10, 2], 1],
 [23, 14, [20, 11], 2],
 [23, 15, [3, 14, 15], 3],
 [28, 15, [19, 25, 6], 4]]

In [20]:
optimality_gap = (upper_bound - machine_list[0][0]) / upper_bound
print("optimality gap =", optimality_gap)

optimality gap = 0.24324324324324315


## Heuristic – HRF (highest ratio first)

### Constant Data – part (b) instance

In [21]:
I = 3 # machine num
J = 10 # job num
machines = range(0, I)
jobs = range(0, J)

In [22]:
# process time for job j
p = [3, 6, 5, 4, 1, 8, 5, 12, 7, 6]

# benefit for job j
b = [5, 8, 4, 6, 3, 7, 6, 9, 5, 8]

# machine capacity
K = 15

### Constant Data – part (c) instance

In [23]:
I = 4 # machine num
J = 25 # job num
machines = range(0, I)
jobs = range(0, J)

In [24]:
# process time for job j
p = [3, 6, 7, 5, 4, 2, 6, 3, 5, 8, 10, 2, 4, 7, 1, 5, 8, 3, 6, 4, 12, 4, 8, 4, 7]

# benefit for job j
b = [5, 8, 12, 4, 6, 6, 3, 7, 6, 15, 9, 5, 8, 10, 1, 5, 3, 7, 12, 14, 5, 8, 9, 8, 10]

# machine capacity
K = 15

In [25]:
# [b, p, index] * job num
job_list = list(zip(b, p, range(1, J+1)))
job_list.sort(key=lambda x: (-x[0]/x[1], x[2]))
print(job_list)

[(14, 4, 20), (6, 2, 6), (5, 2, 12), (7, 3, 8), (7, 3, 18), (8, 4, 13), (12, 6, 19), (8, 4, 22), (8, 4, 24), (15, 8, 10), (12, 7, 3), (5, 3, 1), (6, 4, 5), (10, 7, 14), (10, 7, 25), (8, 6, 2), (6, 5, 9), (9, 8, 23), (1, 1, 15), (5, 5, 16), (9, 10, 11), (4, 5, 4), (3, 6, 7), (5, 12, 21), (3, 8, 17)]


In [26]:
# [current benefit, current process time, assigned job list, index] * machine num
machine_list = [[0, 0, [], i+1] for i in machines]
for j in jobs:
    for i in machines:
        if machine_list[i][1] + job_list[j][1] <= K:
            machine_list[i][0] += job_list[j][0]
            machine_list[i][1] += job_list[j][1]
            machine_list[i][2].append(job_list[j][2])
            machine_list.sort(key=lambda x: (x[0], x[3]))
            print('b =', [machine_list[k][0] for k in machines], 'p =', [machine_list[k][1] for k in machines])
            break

b = [0, 0, 0, 14] p = [0, 0, 0, 4]
b = [0, 0, 6, 14] p = [0, 0, 2, 4]
b = [0, 5, 6, 14] p = [0, 2, 2, 4]
b = [5, 6, 7, 14] p = [2, 2, 3, 4]
b = [6, 7, 12, 14] p = [2, 3, 5, 4]
b = [7, 12, 14, 14] p = [3, 5, 4, 6]
b = [12, 14, 14, 19] p = [5, 4, 6, 9]
b = [14, 14, 19, 20] p = [4, 6, 9, 9]
b = [14, 19, 20, 22] p = [6, 9, 9, 8]
b = [19, 20, 22, 29] p = [9, 9, 8, 14]
b = [19, 20, 29, 34] p = [9, 9, 14, 15]
b = [20, 24, 29, 34] p = [9, 12, 14, 15]
b = [24, 26, 29, 34] p = [12, 13, 14, 15]
b = [25, 26, 29, 34] p = [13, 13, 14, 15]


In [27]:
machine_list

[[25, 13, [8, 19, 1, 15], 4],
 [26, 13, [12, 18, 22, 5], 3],
 [29, 14, [6, 13, 10], 2],
 [34, 15, [20, 24, 3], 1]]

In [28]:
optimality_gap = (upper_bound - machine_list[0][0]) / upper_bound
print("optimality gap =", optimality_gap)

optimality gap = 0.17743830787309037


## testing data

In [29]:
# 生測資
import random
import math

for i in range(1, 21):
    path = 'in' + f'{i:02d}' + '.txt'
    m = random.randint(3,5)
    n = 5 * random.randint(5,9)
    b = [random.choice(range(1, 21)) for i in range(n)]
    p = [random.choice(range(1, 21)) for i in range(n)]
    K = math.ceil(3 * sum(p) / 4 / m)
    with open(path, 'w') as f:
        f.write(f'{m} {n} {K}\n')
        f.write(f'{" ".join(str(x) for x in b)}\n')
        f.write(f'{" ".join(str(x) for x in p)}')

In [30]:
# 讀測資
import os

m = []
n = []
K = []
b = []
p = []

for i in range(1, 21):
    path = 'OR110-2_hw02_data/in' + f'{i:02d}' + '.txt'
    text = []
    with open(path, 'r') as f:
        for line in f.readlines():
            text.append(line)
    temp = text[0].split()
    m.append(int(temp[0]))
    n.append(int(temp[1]))
    K.append(int(temp[2]))
    b.append(list(map(int, text[1].split())))
    p.append(list(map(int, text[2].split())))

### Gurobi

In [31]:
upper_bound = []
for r in range(20):
    machines = range(m[r])
    jobs = range(n[r])
    
    Q4 = Model('Q4')
    
    # decision variable
    # x_ij = 0 if job j is assigned to machine i
    x = {}
    for i in machines:
        x[i] = {}
        for j in jobs:
            x[i][j] = Q4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "x_" + str(i) + "," + str(j))

    w = Q4.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "w")
    
    Q4.update()
    
    
    # Setting the objective function
    Q4.setObjective(w, GRB.MAXIMIZE)
    
    
    # Add constraints
    Q4.addConstrs((w <= quicksum(b[r][j] * x[i][j] for j in jobs) for i in machines), 'least benefit machine')
    Q4.addConstrs((quicksum(x[i][j] for i in machines) <= 1 for j in jobs), 'each job assigned to exactly one machine')
    Q4.addConstrs((quicksum(p[r][j] * x[i][j] for j in jobs) <= K[r] for i in machines), 'machine capacity')

    Q4.update()
    
    
    # optimize
    Q4.optimize()
    upper_bound.append(Q4.objVal)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 51 rows, 136 columns and 408 nonzeros
Model fingerprint: 0x68a81142
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve time: 0.01s
Presolved: 51 rows, 136 columns, 408 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   3.000000e+30   8.000000e+00      0s
      96    1.4670000e+02   0.000000e+00   0.000000e+00      0s

Solved in 96 iterations and 0.01 seconds
Optimal objective  1.467000000e+02
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 53 rows, 181 columns and 544 nonzeros
Model fingerprint: 0xbab0a4e6
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  O

In [32]:
print([round(upper_bound[i], 3) for i in range(len(upper_bound))])

[146.7, 95.279, 54.72, 144.368, 59.82, 115.571, 83.964, 76.575, 85.75, 138.4, 76.067, 154.75, 59.5, 87.333, 115.95, 62.653, 54.632, 87.68, 56.176, 61.25]


### HBF

In [33]:
HBF = []
HBF_gap = []
for r in range(20):
    machines = range(m[r])
    jobs = range(n[r])
    
    job_list = list(zip(b[r], p[r], range(1, n[r]+1)))
    job_list.sort(key=lambda x: (-x[0], x[2]))
    
    machine_list = [[0, 0, [], i+1] for i in machines]
    for j in jobs:
        for i in machines:
            if machine_list[i][1] + job_list[j][1] <= K[r]:
                machine_list[i][0] += job_list[j][0]
                machine_list[i][1] += job_list[j][1]
                machine_list[i][2].append(job_list[j][2])
                machine_list.sort(key=lambda x: (x[0], x[3]))
                if r==0:
                    print('b =', [machine_list[k][0] for k in machines], 'p =', [machine_list[k][1] for k in machines])
                break
    
    HBF.append(machine_list[0][0])
    HBF_gap.append((upper_bound[r] - machine_list[0][0]) / upper_bound[r])

b = [0, 0, 20] p = [0, 0, 14]
b = [0, 19, 20] p = [0, 5, 14]
b = [18, 19, 20] p = [2, 5, 14]
b = [19, 20, 36] p = [5, 14, 11]
b = [20, 36, 37] p = [14, 11, 19]
b = [36, 37, 37] p = [11, 28, 19]
b = [37, 37, 53] p = [28, 19, 22]
b = [37, 53, 54] p = [19, 22, 30]
b = [53, 53, 54] p = [36, 22, 30]
b = [53, 54, 69] p = [22, 30, 39]
b = [54, 69, 69] p = [30, 39, 40]
b = [69, 69, 70] p = [39, 40, 46]
b = [69, 70, 85] p = [40, 46, 43]
b = [70, 84, 85] p = [46, 58, 43]
b = [84, 85, 85] p = [58, 54, 43]
b = [85, 85, 98] p = [54, 43, 77]
b = [85, 98, 98] p = [43, 69, 77]
b = [97, 98, 98] p = [60, 69, 77]
b = [98, 98, 109] p = [69, 77, 69]
b = [98, 109, 110] p = [77, 69, 70]
b = [109, 110, 110] p = [69, 70, 83]
b = [110, 110, 121] p = [70, 83, 89]
b = [110, 121, 121] p = [83, 88, 89]
b = [120, 121, 121] p = [96, 88, 89]
b = [121, 121, 130] p = [88, 89, 101]
b = [121, 130, 130] p = [89, 97, 101]
b = [130, 130, 130] p = [97, 107, 101]
b = [130, 130, 139] p = [107, 101, 117]
b = [130, 137, 139] p = 

In [34]:
print([round(HBF[i], 3) for i in range(len(HBF))])
print([round(HBF_gap[i] * 100, 3) for i in range(len(HBF_gap))])
print(sum(HBF_gap) / len(HBF_gap))

[144, 89, 50, 136, 49, 108, 76, 70, 80, 137, 63, 146, 55, 78, 108, 59, 54, 76, 54, 55]
[1.84, 6.591, 8.626, 5.797, 18.088, 6.551, 9.485, 8.586, 6.706, 1.012, 17.178, 5.654, 7.563, 10.687, 6.856, 5.83, 1.157, 13.321, 3.874, 10.204]
0.07780333783677044


### HRF

In [35]:
HRF = []
HRF_gap = []
for r in range(20):
    machines = range(m[r])
    jobs = range(n[r])
    
    job_list = list(zip(b[r], p[r], range(1, n[r]+1)))
    job_list.sort(key=lambda x: (-x[0]/x[1], x[2]))
    
    machine_list = [[0, 0, [], i+1] for i in machines]
    for j in jobs:
        for i in machines:
            if machine_list[i][1] + job_list[j][1] <= K[r]:
                machine_list[i][0] += job_list[j][0]
                machine_list[i][1] += job_list[j][1]
                machine_list[i][2].append(job_list[j][2])
                machine_list.sort(key=lambda x: (x[0], x[3]))
                break
    HRF.append(machine_list[0][0])
    HRF_gap.append((upper_bound[r] - machine_list[0][0]) / upper_bound[r])

In [36]:
print([round(HRF[i], 3) for i in range(len(HRF))])
print([round(HRF_gap[i] * 100, 3) for i in range(len(HRF_gap))])
print(sum(HRF_gap) / len(HRF_gap))

[144, 92, 51, 140, 55, 111, 81, 71, 80, 136, 72, 150, 53, 82, 109, 58, 51, 82, 50, 57]
[1.84, 3.442, 6.798, 3.026, 8.058, 3.956, 3.53, 7.28, 6.706, 1.734, 5.346, 3.069, 10.924, 6.107, 5.994, 7.426, 6.649, 6.478, 10.995, 6.939]
0.05814865611454632


### HRF modified (break tie by smallest processing time)

In [37]:
HRF_new = []
HRF_new_gap = []
for r in range(20):
    machines = range(m[r])
    jobs = range(n[r])
    
    job_list = list(zip(b[r], p[r], range(1, n[r]+1)))
    # first sort by ratio (descend), then by processing time (ascend), and finally by index (ascend)
    job_list.sort(key=lambda x: (-x[0]/x[1], x[1], x[2]))
    
    machine_list = [[0, 0, [], i+1] for i in machines]
    for j in jobs:
        for i in machines:
            if machine_list[i][1] + job_list[j][1] <= K[r]:
                machine_list[i][0] += job_list[j][0]
                machine_list[i][1] += job_list[j][1]
                machine_list[i][2].append(job_list[j][2])
                machine_list.sort(key=lambda x: (x[0], x[3]))
                break
    HRF_new.append(machine_list[0][0])
    HRF_new_gap.append((upper_bound[r] - machine_list[0][0]) / upper_bound[r])

In [38]:
print([round(HRF_new[i], 3) for i in range(len(HRF_new))])
print([round(HRF_new_gap[i] * 100, 3) for i in range(len(HRF_new_gap))])
print(sum(HRF_new_gap) / len(HRF_new_gap))

[144, 92, 51, 143, 55, 114, 78, 72, 80, 137, 69, 150, 53, 82, 109, 58, 51, 85, 50, 57]
[1.84, 3.442, 6.798, 0.948, 8.058, 1.36, 7.103, 5.975, 6.706, 1.012, 9.29, 3.069, 10.924, 6.107, 5.994, 7.426, 6.649, 3.057, 10.995, 6.939]
0.056845184636818445
